# DCGAN
- kerasは、最上位のモジュールのみをimportして使うようにしました。
- プログラムは長くなってしまいますが、kerasにまだあまり精通していない場合はこちらのほうが読みやすいんじゃないかと。

In [0]:
import keras
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [0]:
from google.colab import drive
drive.mount('drive')

### 🐍mnistデータ読み込む🐍
- 読み込んだ後に画像の画素値を-1～1に正規化した

In [0]:
(X_train, _), (_, _) = keras.datasets.mnist.load_data()
X_train = (X_train.astype(np.float32) - 127.5) / 127.5
X_train = np.expand_dims(X_train, axis=3)

batch_size = 256

half_batch = int(batch_size / 2)

### 🐍最適化関数はAdamに🐍
- なんでAdamにしたか？[ここ](https://www.renom.jp/ja/notebooks/tutorial/basic_algorithm/adam/notebook.html)参照してください。
- でも現在はAdaboundっていう、Adamの安定し辛さを改善してくれたのが出てるんですよね…。使いたいなぁ。

In [0]:
z_dim = 100

img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels)

optimizer = keras.optimizers.Adam(0.0002, 0.5)

### 🐍Generatorを設計🐍

In [0]:
noise_shape = (z_dim,)
generator = keras.models.Sequential()

generator.add(keras.layers.Dense(1024, input_shape=noise_shape))
generator.add(keras.layers.normalization.BatchNormalization())
generator.add(keras.layers.Activation('relu'))
generator.add(keras.layers.Dense(128*7*7))
generator.add(keras.layers.normalization.BatchNormalization())
generator.add(keras.layers.Activation('relu'))
generator.add(keras.layers.Reshape((7,7,128), input_shape=(128*7*7, )))
generator.add(keras.layers.UpSampling2D((2, 2)))
generator.add(keras.layers.Conv2D(64, (5, 5), padding='same'))
generator.add(keras.layers.normalization.BatchNormalization())
generator.add(keras.layers.Activation('relu'))
generator.add(keras.layers.UpSampling2D((2, 2)))
generator.add(keras.layers.Conv2D(1, (5, 5), padding='same'))
generator.add(keras.layers.Activation('tanh'))

### 🐍Discriminatorを設計🐍

In [0]:
img_shape = (img_rows, img_cols, channels)
discriminator = keras.models.Sequential()

discriminator.add(keras.layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=img_shape))
discriminator.add(keras.layers.advanced_activations.LeakyReLU(0.2))
discriminator.add(keras.layers.Conv2D(128, (5, 5), strides=(2, 2)))
discriminator.add(keras.layers.advanced_activations.LeakyReLU(0.2))
discriminator.add(keras.layers.Flatten())
discriminator.add(keras.layers.Dense(256))
discriminator.add(keras.layers.advanced_activations.LeakyReLU(0.2))
discriminator.add(keras.layers.Dropout(0.5))
discriminator.add(keras.layers.Dense(1))
discriminator.add(keras.layers.Activation('sigmoid'))

In [0]:
discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

### 🐍combinedを設計🐍
- GeneratorとDiscriminatorをつなぎ合わせたものを作成しておく
- Generatorを学習させるときは、Generator単体ではなく、このCombinedを使って学習を行う

In [0]:
discriminator.trainable = False
combined = keras.models.Sequential([generator, discriminator])
combined.compile(loss='binary_crossentropy', optimizer=optimizer)

### 🐍学習🐍
- 規定回数ごとに画像を保存している
- ちょっとあとでもう少しわかりやすくします
- 最終的には、Discriminatorの正答率が50%になるくらいまで、Generatorに成長してもらいたい

In [0]:
for epoch in range(15001):
    
    # 以下、Discriminator学習
    # Generatorによる偽データの生成
    noise = np.random.normal(0, 1, (half_batch, z_dim))
    gen_imgs = generator.predict(noise)
    
    # バッチサイズの半数を教師データからピックアップ
    idx = np.random.randint(0, X_train.shape[0], half_batch)
    imgs = X_train[idx]
    
    # discriminator学習
    d_loss_real = discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
    d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
    
    # それぞれの損失関数を平均
    d_loss_mean = np.add(d_loss_real, d_loss_fake) * 0.5
    
    # 以下、Generator学習
    noise = np.random.normal(0, 1, (batch_size, z_dim))
    g_loss = combined.train_on_batch(noise, np.ones((batch_size, 1)))
    
    if epoch % 500 == 0:
        
        row = 5
        columns = 5
        save_gen_imgs = gen_imgs[: row * columns]
        save_gen_imgs = 0.5 * save_gen_imgs + 0.5
        fig, axes = plt.subplots(row, columns)

        for r in range(row):
            for c in range(columns):
                axes[r, c].imshow(save_gen_imgs[r * 5 + c, :, :, 0], cmap='gray')
                axes[r, c].axis('off')

        plt.savefig('drive/My Drive/GitHub/DCGAN_mnist/savefig/{}.png'.format(str(epoch)))
        plt.close()
        
        print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss_mean[0], 100*d_loss_mean[1], g_loss))